SFAM is a supervised algorithm by definition, so we use it to map a set of features to a set of supervisory labels.
We will do so by training and testing on the ubiquitous Iris dataset and seeing how well the SFAM module generalizes the data.

We begin with importing AdaptiveResonance for the ART modules and MLDatasets for some data utilities.

In [1]:
using AdaptiveResonance # ART
using MLDatasets        # Iris dataset
using MLDataUtils       # Shuffling and splitting
using Printf            # Formatted number printing

We will download the Iris dataset for its small size and benchmark use for clustering algorithms.

In [2]:
Iris.download(i_accept_the_terms_of_use=true)
features, labels = Iris.features(), Iris.labels()

([5.1 4.9 … 6.2 5.9; 3.5 3.0 … 3.4 3.0; 1.4 1.4 … 5.4 5.1; 0.2 0.2 … 2.3 1.8], ["Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa", "Iris-setosa"  …  "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica", "Iris-virginica"])

Because the MLDatasets package gives us Iris labels as strings, we will use the `MLDataUtils.convertlabel` method with the `MLLabelUtils.LabelEnc.Indices` type to get a list of integers representing each class:

In [3]:
labels = convertlabel(LabelEnc.Indices{Int}, labels)
unique(labels)

3-element Array{Int64,1}:
 1
 2
 3

Next, we will create a train/test split with the `MLDataUtils.stratifiedobs` utility:

In [4]:
(X_train, y_train), (X_test, y_test) = stratifiedobs((features, labels))

(([6.7 5.6 … 4.9 6.7; 3.0 3.0 … 2.4 3.0; 5.2 4.1 … 3.3 5.0; 2.3 1.3 … 1.0 1.7], [3, 2, 3, 3, 2, 1, 2, 2, 1, 1  …  2, 2, 1, 3, 3, 1, 3, 3, 2, 2]), ([5.7 6.5 … 7.7 6.4; 2.6 3.0 … 3.8 3.2; 3.5 5.8 … 6.7 5.3; 1.0 2.2 … 2.2 2.3], [2, 3, 2, 2, 1, 3, 3, 3, 2, 1  …  1, 2, 2, 3, 2, 3, 3, 1, 3, 3]))

Now, we can create our SFAM module.
We'll do so with the default contstructor, though the module itself has many options that can be altered during instantiation.

In [5]:
# Create the SFAM module
art = SFAM()

# Change the match tracking parameter after instantiation
art.opts.epsilon = 1e-2

0.01

We can train the model in batch mode upon the data and supervisory labels.
We do so by directly passing the integer vector of labels to the training method.
Just as in other modules, we can extract the SFAM's prescribed labels from the training method, which should match up to the training labels as we will see later.

In [6]:
# Train in supervised mode by directly passing the labels.
y_hat_train = train!(art, X_train, y_train)
println("Training labels: ",  size(y_hat_train), " ", typeof(y_hat_train))

[ Info: Training SFAM
0.0%┣                                               ┫ 0/105 [00:00<00:-5, 0s/it]
Ep: 1, ID: 1, Cat: 0 1.0%┣▏                    ┫ 1/105 [00:00<Inf:Inf, InfGs/it]
Ep: 1, ID: 105, Cat: 16 100.0%┣████████████████┫ 105/105 [00:00<00:00, 1.9kit/s]
Training labels: (105,) Array{Int64,1}


We can classify the testing data to see how we generalize.
At the same time, we can see the effect of getting the best-matching unit in the case of complete mismatch (see the docs on Mismatch vs. BMU)

In [7]:
# Classify both ways
y_hat = AdaptiveResonance.classify(art, X_test)
y_hat_bmu = AdaptiveResonance.classify(art, X_test, get_bmu=true)

# Check the shape and type of the output labels
println("Testing labels: ",  size(y_hat), " ", typeof(y_hat))
println("Testing labels with bmu: ",  size(y_hat_bmu), " ", typeof(y_hat_bmu))

[ Info: Testing SFAM
0.0%┣                                                ┫ 0/45 [00:00<00:-2, 0s/it]
Ep: 1, ID: 1, Cat: 16 2.2%┣▌                    ┫ 1/45 [00:00<Inf:Inf, InfGs/it]
Ep: 1, ID: 45, Cat: 16 100.0%┣████████████████████┫ 45/45 [00:00<00:00, 843it/s]
[ Info: Testing SFAM
0.0%┣                                                ┫ 0/45 [00:00<00:-2, 0s/it]
Ep: 1, ID: 1, Cat: 16 2.2%┣▌                    ┫ 1/45 [00:00<Inf:Inf, InfGs/it]
Ep: 1, ID: 45, Cat: 16 100.0%┣████████████████████┫ 45/45 [00:00<00:00, 847it/s]
Testing labels: (45,) Array{Int64,1}
Testing labels with bmu: (45,) Array{Int64,1}


Finally, we can calculate the performances (number correct over total) of the model upon all three regimes:
1. Training data
2. Testing data
2. Testing data with `get_bmu=true`

In [8]:
# Calculate performance on training data, testing data, and with get_bmu
perf_train = performance(y_hat_train, y_train)
perf_test = performance(y_hat, y_test)
perf_test_bmu = performance(y_hat_bmu, y_test)

# Format each performance number for comparison
@printf "Training performance: %.4f\n" perf_train
@printf "Testing performance: %.4f\n" perf_test
@printf "Best-matching unit testing performance: %.4f\n" perf_test_bmu

Training performance: 1.0000
Testing performance: 0.9556
Best-matching unit testing performance: 0.9556


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*